In [ ]:
from ipaddress import ip_address

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from matplotlib.colors import LogNorm
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
algos = {
    'unsigned': 0,
    'dilithium2': 18,
    'ecdsa256': 13,
    'ed25519': 15,
    'falcon512': 17,
    'rsasha256': 8,
    'sphincs-sha256-128s': 19,
    'xmssmt-sha256-h40-4': 21,
    'xmssmt-sha256-h40-8': 21,
}
rcodes = ['NOERROR', 'NXDOMAIN', 'SERVFAIL', 'REFUSED', 'FORMERR']
df = pd.read_csv("results.csv")
df.rename(columns={"rtt": "response_time"}, inplace=True)
df['algo'] = df['algo'].transform(lambda x: f'{algos[x.rstrip("3")]:02}_{x}')
df['rcode'] = pd.Categorical(df['rcode'], rcodes)
df['private_dst'] = df['dst_addr'].apply(lambda x: x and ip_address(x).is_private)
for column in ["is_nx", "correct_response", "tcp", "do", "ad", "bad_unsigned", "bad_rsa"]:
    df[column] = df[column].map({'t': True, 'f': False}).astype("boolean")
df.head()
df.tail()

In [ ]:
df = df[df.private_dst == False]

In [ ]:
done = False
for tag, cond in {'good-unsigned': (df.bad_unsigned == False), 'good-rsa': (df.bad_rsa == False)}.items():
    if done: break
    for vendor in ['pdns', 'bind9']:
        if done: break
        with PdfPages(f'results_{vendor}_{tag}.pdf') as pdf_pages:
            for x in ['rcode', 'correct_response', 'ad', 'response_time']:
                if done: break
                for is_nx in [False, True]:
                    if done: break
                    kwargs = {}
                    if df[x].dtype == 'boolean':
                        kwargs['discrete'] = True
                    #    kwargs['bins'] = 2
                    #    kwargs['binrange'] = ((-.5, 1.5), None)
                    if x == 'response_time':
                        kwargs['binrange'] = ((0, 10000), None)
                    #    kwargs['log_scale'] = (True, False)
                    if x == 'rcode':
                        kwargs['bins'] = (rcodes, None)
                    g = sns.displot(df[(df.is_nx == is_nx) & cond & (df.vendor == vendor)].sort_values(['algo', x]), x=x, y='algo', cbar=True, cmap='viridis_r', norm=LogNorm(), vmin=None, vmax=None, row='tcp', col='do', aspect=1.5, **kwargs)
                    g.fig.subplots_adjust(top=0.9)
                    g.fig.suptitle(f'{vendor=}, {is_nx=}, {tag}')
                    if x != 'response_time':
                        for row in g.axes:
                            if done: break
                            for ax in row:
                                x_dim = ax.collections[0].get_coordinates().shape[1] - 1
                                y_dim = ax.collections[0].get_coordinates().shape[0] - 1
                                hist_array = ax.collections[0].get_array().reshape([y_dim, x_dim])
                                counts = {k: hist_array[k].sum() for k in range(y_dim)}
                                for k, v in enumerate(ax.collections[0].get_array().ravel()):
                                    if not np.ma.is_masked(v):
                                        fraction = v/counts[k//x_dim]
                                        ax.annotate(f'{fraction*100:.3g}%', xy=(k%x_dim, k//x_dim), ha='center', color='white' if fraction >= 0.005 else 'black')
                    pdf_pages.savefig(plt.gcf())